In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'stable-diffusion-image-to-prompts:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F45917%2F5024308%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240806%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240806T193113Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6a6f2710ad1fd55bf2ab1bb8729db6dbf2967e4ac71f820b57f0550e50f031755cd7451d5627e1155160691df1f09930812dba00b44e9cebebe7c11e6e5c34408bb678fc2808f43919bd7c4fe4babd376b91b2a9294f3eff0948abb81947b2c72f99ddece351e9ff3e5c9f1747c36adc60a082016452980e752626c15cded67a757c8006bece256a5f925e5390e2086f8881f20a1d1883c3d09917b6e234ea8627125eecb7b01656d083e45efdc42431f8052acfc79c0232a5c8b062bd5624f1bcf67f9255391f87190fd3fb3567a68410fd84fdd94f10ccb1ac954ac517b1669254f3eea7b8fb99d635b0527066a2407d4ac77f5fbbaae7f35233dd4f6aa885,sentence-transformers-222:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2893282%2F4988409%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240806%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240806T193113Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1f9bd9aa1544593fbdf67e78531983286e9b8288461fe62199f418cc3c0369386ab184c4712cc67c842b09651251f095eceadaa4d9b76a3089ad42d07611cfc676a9a1f9290267f1100e1b71f6fda03c9892b6c3a281ed802feff539e9a8f8b9ec7bbb71da749f9ba1eea1d726f58ecbdf7e144b82f430cce482f32913c1fc802cd5a36a0442fb2a6e3d8b985db2b2dd41e8295f7a14453efd86efbdb4c30f8460e369862e757c73164d7f6fc5b2b0a0d455c03d392a0f5821b6e2867fd385c16cce0b6ba713fe9a2411db8668eabf7ec1bd82ca8b3f94392b204cbcbd4ee6bc6d408aa6931d0e7f522ebe16a837862a5d9ed06cd496595f7b2a031ae5fe04c6,open-clip-wheels:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2908135%2F5011787%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240806%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240806T193113Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D29686fd8cc23ba8854eac3e259875803202f0a27a20895dfb1b3632db5959997bdb1e95456b8aebdeaf53176deac0343a977aa365556075670af4c659f4757a6bd44f838ed1e9208f7403e00b3440139cb1b599de2d0dd67db2e900ab7965f67a0bfcf6290602211a4cf7b9ebfb1b64ec083be25ab12c87d91fb1e252c3eb4149a64f9238d1711e6afb082f7921c69993bbcf9c578b0b6be763e7b5fa3e19a81bccd4e80ed6680455da46f1e7f189dadc3950064d45f8eefe6a4d99bfca2fc2e36bc3a4f4230ef2ce7382a858572d26df5c640ee4eda5bb62ac60aaa8baeca24e6d688b2467a433f660ed817b794101452d790f40e24ed7b8cd6232222c1dfcd,open-clip-models:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2908357%2F5012196%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240806%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240806T193113Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da9e9f1babfe8ba669f864217a718bd693fbf103742a067da2b3e2f3b13137682653a0e163bbc0f8b9b2b3cd66f0c7787fe0c5d0a45411c674ba7f5de4074336e3ccd6ee441ccf241b22122c22c0a2b5e0de353df1e7ec332bf4d8103cee1dd8aea0c553eaadcb88ca94fa67b317e7530e84e2516bf255ffd3b7cfbe8b1bf75672577ad52edb1db37daad00b087b884f4e575407ffc160da47c3410eb83e7ff6b37a3b23c08ea3fac3e4d2280906b0530aeaf3e940d739d6a71e3d784b9ad542e96d4417266bd4c01cccaa993ad2e11730d389395fc7ee6c7f382fdba97c623078866b3206dbea2c11e66ec1695bd323ca578086f8e4753a6a36a778979b63230'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
!pip install --upgrade diffusers transformers -q

In [ ]:
from pathlib import Path
import tqdm

import torch
import pandas as pd
import numpy as np
from diffusers import StableDiffusionPipeline
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt

from kaggle_secrets import UserSecretsClient

In [ ]:
class IMGCFG:
    device = "cuda"
    seed = 42
    generator = torch.Generator(device).manual_seed(seed)
    image_gen_steps = 35
    image_gen_model_id = "stabilityai/stable-diffusion-2"
    image_gen_size = (512, 512)
    image_gen_guidance_scale = 9
    prompt_gen_model_id = "gpt2"
    prompt_dataset_size = 6
    prompt_max_length = 12

In [ ]:
secret_hf_token = UserSecretsClient().get_secret("secret_hf_token")

In [ ]:
image_gen_model = StableDiffusionPipeline.from_pretrained(
    IMGCFG.image_gen_model_id, torch_dtype=torch.float16,
    revision="fp16", use_auth_token=secret_hf_token, guidance_scale=9
)
image_gen_model = image_gen_model.to(IMGCFG.device)

In [ ]:
prompts_gen_model = pipeline('text-generation', model='gpt2')

In [ ]:
def generate_image(prompt, model):
    image = model(
        prompt, num_inference_steps=IMGCFG.image_gen_steps,
        generator=IMGCFG.generator,
        guidance_scale=IMGCFG.image_gen_guidance_scale
    ).images[0]

    image = image.resize(IMGCFG.image_gen_size)
    return image

In [ ]:
wheels_path = "/kaggle/input/open-clip-wheels/open_clip_wheels"
open_clip_whl_path = f"{wheels_path}/open_clip_torch-2.14.0-py3-none-any.whl"

In [ ]:
!pip install --no-index --find-links $wheels_path $open_clip_whl_path -q

In [ ]:
import os
import sys
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import torch
import open_clip


sys.path.append('../input/sentence-transformers-222/sentence-transformers')
from sentence_transformers import SentenceTransformer, models

comp_path = Path('/kaggle/input/stable-diffusion-image-to-prompts/')

In [ ]:
class TXTCFG:
    device = "cuda"
    seed = 42
    embedding_length = 384
    sentence_model_path = \
        "/kaggle/input/sentence-transformers-222/all-MiniLM-L6-v2"
    model_name = \
        "coca_ViT-L-14"
    model_checkpoint_path = \
        "/kaggle/input/open-clip-models/mscoco_finetuned_CoCa-ViT-L-14-laion2B-s13B-b90k.bin"

In [ ]:
st_model = SentenceTransformer(TXTCFG.sentence_model_path)

In [ ]:
model = open_clip.create_model(TXTCFG.model_name)
open_clip.load_checkpoint(model, TXTCFG.model_checkpoint_path)

In [ ]:
transform = open_clip.image_transform(
    model.visual.image_size,
    is_train = False,
    mean = getattr(model.visual, 'image_mean', None),
    std = getattr(model.visual, 'image_std', None),
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

In [ ]:
def generate_prompt(img):
    img = transform(img).unsqueeze(0)
    with torch.no_grad(), torch.cuda.amp.autocast():
        generated = model.generate(img.to(device))
    return open_clip.decode(
            generated[0]
            ).split("<end_of_text>")[0].replace("<start_of_text>", "").rstrip(" .,")

In [ ]:
def plot_image(image: np.ndarray, title: str) -> None:
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.axis('off')
    plt.annotate(
        title, xy=(1.05, 0.5), xycoords='axes fraction',
        ha='left', va='center', fontsize=16, rotation=0, color="#00786b"
    )
    plt.show();

In [ ]:
# prompt = "Kaggle employee Phil at a donut shop ordering all the best donuts, \
#          with a speech bubble that proclaims 'Donuts. It\'s what's for dinner!'"

prompt = "temple in ruines, forest, stairs, columns, cinematic, detailed, atmospheric, \
            epic, concept art, Matte painting, background, mist, photo-realistic, \
            concept art, volumetric light, cinematic epic + rule of thirds octane render, \
            8k, corona render, movie concept art, octane render, cinematic, \
            trending on artstation, movie concept art, cinematic composition, \
            ultra-detailed, realistic , hyper-realistic , volumetric lighting, \
            8k –ar 2:3 –test –uplight"


for _ in range(50):
    image = generate_image(prompt, image_gen_model)
    prompt = generate_prompt(image)
    plot_image(image, prompt)